In [1]:
import findspark
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from operator import add
import math
import pandas as pd

#Creating Spark Context
sc = SparkContext.getOrCreate()
#Creating Spark Session
spark = SparkSession(sc)
#Readind data set(csv file),dropping null values and selecting only tabs and views column
tags_views = spark.read.csv("file:///C:\\Users\\Hp\\A BDA PROJECT\\Youtube-Big-data-Analytics-using-Spark-master\\USvideos.csv", inferSchema = True, header = True).dropna().select("tags","views")
#Mapping by splitting tags with "|" character,
def tags_split(x):
    tags=x["tags"].split("|")
    result=[]
    for every in tags:
        #Associating view count as counter and applying log on view count because views count will be really large
        if not str(x["views"]).isdigit() or every==None:
            continue
        if x["views"]!=0:
            result.append((every.strip("\"").lower(),math.log(x["views"])))#Stripping unnecessary characters
    return tuple(result)
rdd1=tags_views.rdd.flatMap(tags_split).reduceByKey(add)#reduce by similar tags and adding its view count
#Top Tags are queried from RDD by Sorting RDD's in descending order of view count
toptags=rdd1.takeOrdered(35, key = lambda x: -x[1])
df=spark.createDataFrame(toptags)
#Writing back to Disk
print(df.head(5))
pandasDF = df.toPandas()
print(pandasDF)
pandasDF.to_csv("E:\\trending.csv")
# df.repartition(1).write.csv(path="file:///D:\\trending.csv")
#Spark session
spark.stop()



RuntimeError: Java gateway process exited before sending its port number